In [ ]:
import pandas as pd

import matplotlib

import matplotlib.pyplot as plt

import numpy as np

import seaborn as sns

import sklearn

import warnings

warnings.filterwarnings('ignore')

import imblearn

%matplotlib inline

In [ ]:
# 42 columns present in the dataset 

columns = ["duration","protocol_type","service","flag","src_bytes","dst bytes","land","wrong fragment","urgent","hot" ,"num failed logins", "logged_in","number compromised","root shell","su attempted","num root","num file creation","num shells","num access files","num outbound commands","is_host_login","is_guest_login","count","srv_count","serror_rate","srv serror rate","rerror rate","srv rerror rate count","dst host same srv_rate", "dst_host_diff_srv_rate","dst_host same src_port rate","dst host sry diff host rate","dst host serror rate","dst host srv serror rate","dst host rerror rate","dst_host_srv_rerror_rate","attack","last_flag"]




dfkdd_train = pd.read_table("../input/intrusion-detection/KDDTrain.txt", sep=",", names=columns)
dfkdd_train = dfkdd_train.iloc[:,:-1]


dfkdd_test = pd.read_table("../input/intrusion-detection/KDDTest.txt", sep=",", names=columns) 
dfkdd_test = dfkdd_test.iloc[:,:-1]
dfkdd_train.head()

In [ ]:
print('Test set dimension: {} rows, {} columns'.format(dfkdd_test.shape[0], dfkdd_test.shape[1]))

In [ ]:

mapping = {'ipsweep': 'Probe','satan': 'Probe','nmap': 'Probe','portsweep': 'Probe','saint': 'Probe','mscan': 'Probe',
        'teardrop': 'DoS','pod': 'DoS','land': 'DoS','back': 'DoS','neptune': 'DoS','smurf': 'DoS','mailbomb': 'DoS',
        'udpstorm': 'DoS','apache2': 'DoS','processtable': 'DoS',
        'perl': 'U2R','loadmodule': 'U2R','rootkit': 'U2R','buffer_overflow': 'U2R','xterm': 'U2R','ps': 'U2R',
        'sqlattack': 'U2R','httptunnel': 'U2R',
        'ftp_write': 'R2L','phf': 'R2L','guess_passwd': 'R2L','warezmaster': 'R2L','warezclient': 'R2L','imap': 'R2L',
        'spy': 'R2L','multihop': 'R2L','named': 'R2L','snmpguess': 'R2L','worm': 'R2L','snmpgetattack': 'R2L',
        'xsnoop': 'R2L','xlock': 'R2L','sendmail': 'R2L',
        'normal': 'Normal'
        }

In [ ]:
dfkdd_train['attack_class'] = dfkdd_train['attack'].apply(lambda x: mapping[x])
dfkdd_test['attack_class'] = dfkdd_test['attack'].apply(lambda x: mapping[x])

In [ ]:
dfkdd_train.drop(['attack'], axis=1, inplace=True)
dfkdd_test.drop(['attack'], axis=1, inplace=True)


In [ ]:
dfkdd_train['attack_class'].value_counts()

In [ ]:
attack_class_freq_train = dfkdd_train[['attack_class']].apply(lambda x: x.value_counts())
attack_class_freq_test = dfkdd_test[['attack_class']].apply(lambda x: x.value_counts())
attack_class_freq_train['frequency_percent_train'] = round((100 * attack_class_freq_train / attack_class_freq_train.sum()),2)
attack_class_freq_test['frequency_percent_test'] = round((100 * attack_class_freq_test / attack_class_freq_test.sum()),2)

attack_class_dist = pd.concat([attack_class_freq_train,attack_class_freq_test], axis=1) 
attack_class_dist

In [ ]:
plot = attack_class_dist[['frequency_percent_train', 'frequency_percent_test']].plot(kind="bar")
plt.figure(figsize=(20,20));
plot.set_title("Attack Class Distribution", fontsize=22)
plot.grid(color='lightgray', alpha=0.4)
plot.legend()

In [ ]:
#Feature part

In [ ]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


cols = dfkdd_train.select_dtypes(include=['float64','int64']).columns
sc_train = scaler.fit_transform(dfkdd_train.select_dtypes(include=['float64','int64']))
sc_test = scaler.fit_transform(dfkdd_test.select_dtypes(include=['float64','int64']))


sc_traindf = pd.DataFrame(sc_train, columns = cols)
sc_testdf = pd.DataFrame(sc_test, columns = cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()


cattrain = dfkdd_train.select_dtypes(include=['object']).copy()
cattest = dfkdd_test.select_dtypes(include=['object']).copy()

traincat = cattrain.apply(encoder.fit_transform)
testcat = cattest.apply(encoder.fit_transform)


enctrain = traincat.drop(['attack_class'], axis=1)
enctest = testcat.drop(['attack_class'], axis=1)

cat_Ytrain = traincat[['attack_class']].copy()
cat_Ytest = testcat[['attack_class']].copy()

In [ ]:
from imblearn.over_sampling import RandomOverSampler 
from collections import Counter


sc_traindf = dfkdd_train.select_dtypes(include=['float64','int64'])
refclasscol = pd.concat([sc_traindf, enctrain], axis=1).columns
refclass = np.concatenate((sc_train, enctrain.values), axis=1)
X = refclass


c, r = cat_Ytest.values.shape
y_test = cat_Ytest.values.reshape(c,)

c, r = cat_Ytrain.values.shape
y = cat_Ytrain.values.reshape(c,)


ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_sample(X, y)
print('Original dataset shape {}'.format(Counter(y)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

In [ ]:
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier 

#Train Kneighnbors
KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
KNN_Classifier.fit(X_res, y_res);

#Train Logistic
LGR_Classifier = LogisticRegression(n_jobs=-1, random_state=0)
LGR_Classifier.fit(X_res, y_res);

#Train Gaussian
BNB_Classifier = BernoulliNB()
BNB_Classifier.fit(X_res, y_res)

#Train Decision 
DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
DTC_Classifier.fit(X_res, y_res);

In [ ]:
from sklearn import metrics
models =[]
#models.append
models.append(('Naive Baye Classifier', BNB_Classifier))
models.append(('Decision Tree Classifier', DTC_Classifier))
models.append(('KNeighborsClassifier', KNN_Classifier))
models.append(('LogisticRegression',LGR_Classifier))

acc=[]
for i, v in models:
  accuracy= metrics.accuracy_score(y_res, v.predict(X_res))
  acc.append(accuracy)
print("Accuracy: ",acc)

In [ ]:
pd.DataFrame({"Model":["Naive Bayes","Decision Tree","K Neighbors","Logistic Regresssion"],"Accuracy":acc})

In [ ]:
import matplotlib.pyplot as plt

plt.barh(["Naive Bayes","Decision Tree","K Neighbors","Logistic Regresssion"],acc)
plt.xlabel("Accuracy")